In [0]:
entity = 'messages'

In [0]:
df_messages_bronze_history = spark.sql("DESCRIBE HISTORY slack_etl.bronze.messages LIMIT 1")
messages_bronze_latest_commit_version = df_messages_bronze_history.select('version').orderBy('version', ascending=False).limit(1).collect()[0].asDict()['version']

In [0]:
df_messages_bronze = spark.readStream.table('slack_etl.bronze.messages')


In [0]:
from pyspark.sql.functions import col, explode

In [0]:
df_messages_attachments_silver = (
    df_messages_bronze.withColumn('attachment',explode(col('attachments')))
    .select(col('attachment.id').alias('attachment_id'),'attachment.mimetype','attachment.size','channel.id','client_msg_id','thread.reply_count','thread.thread_ts', 'thread.participants')
                                      
                                  )

In [0]:
df_messages_threads_silver = (
    df_messages_bronze
    .select('channel.id','client_msg_id','thread.reply_count','thread.thread_ts', 'thread.participants')
                                      
                                  )

In [0]:
df_messages_reactions_silver = (
    df_messages_bronze.withColumn('reaction',explode(col('reactions')))
    .select('reaction.count','reaction.name', 'channel.id','client_msg_id','thread.reply_count','thread.thread_ts')
                                      
                                  )

In [0]:
df_messages_reactions_silver.writeStream.format('delta').option('checkpointLocation',f'/Volumes/slack_etl/silver/metadata/checkpoints/{entity}/reactions/').outputMode('append').trigger(availableNow=True).toTable(f'slack_etl.silver.message_reactions')

In [0]:
df_messages_attachments_silver.writeStream.format('delta').option('checkpointLocation',f'/Volumes/slack_etl/silver/metadata/checkpoints/{entity}/attachments/').outputMode('append').trigger(availableNow=True).toTable(f'slack_etl.silver.message_attachments')

In [0]:
df_messages_threads_silver.writeStream.format('delta').option('checkpointLocation',f'/Volumes/slack_etl/silver/metadata/checkpoints/{entity}/threads/').outputMode('append').trigger(availableNow=True).toTable(f'slack_etl.silver.message_threads')